In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

num_input = 28
timesteps = 28
num_hidden = 128
num_classes = 10

X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

weights = {
    'out': tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [3]:
def BiRNN(x, weights, biases):
    x = tf.unstack(x, timesteps, 1)
    
    # Define a lstm cell with tensorflow
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias= 1.0)
    
    # Get lstm cell output
    try: 
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell,
                                                    x, dtype= tf.float32)
    except Exception: 
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lst_bw_cell, x,
                                              dtype=tf.float32)
    
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits = BiRNN(X, weights, biases)
prediction = tf.nn.softmax(logits)


# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [4]:
with tf.Session() as sess:
    sess.run(init)
    
    # Keep training until reach max iterations
    
    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss & accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, 
                                                                Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                 "{:.4f}".format(loss) + ", Training Accuracy= " + \
                 "{:.3f}".format(acc))
            
    print("Optimization Finished!")
    
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
         sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.8870, Training Accuracy= 0.117
Step 200, Minibatch Loss= 2.1020, Training Accuracy= 0.359
Step 400, Minibatch Loss= 1.9498, Training Accuracy= 0.445
Step 600, Minibatch Loss= 1.9119, Training Accuracy= 0.352
Step 800, Minibatch Loss= 1.7351, Training Accuracy= 0.508
Step 1000, Minibatch Loss= 1.6250, Training Accuracy= 0.492
Step 1200, Minibatch Loss= 1.4952, Training Accuracy= 0.531
Step 1400, Minibatch Loss= 1.5300, Training Accuracy= 0.508
Step 1600, Minibatch Loss= 1.4156, Training Accuracy= 0.625
Step 1800, Minibatch Loss= 1.4986, Training Accuracy= 0.508
Step 2000, Minibatch Loss= 1.3860, Training Accuracy= 0.547
Step 2200, Minibatch Loss= 1.2126, Training Accuracy= 0.664
Step 2400, Minibatch Loss= 1.1990, Training Accuracy= 0.672
Step 2600, Minibatch Loss= 1.2072, Training Accuracy= 0.609
Step 2800, Minibatch Loss= 1.0501, Training Accuracy= 0.656
Step 3000, Minibatch Loss= 1.1402, Training Accuracy= 0.625
Step 3200, Minibatch Loss= 0.9821, Training Acc